In [6]:
import json
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import glob
import json
import pandas as pd

In [8]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/bybit/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = filename.replace('backtest_result.txt','live_config.json')
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']
        if 'static_grid' not in config_type:
            continue

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/bybit','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_706988/2290462972.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_706988/2290462972.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_706988/2290462972.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_706988/2290462972.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\ADAUSDT\plots\2022-03-24T141758\backtest_resu...,5.5,ADAUSDT,True,10.6500,0.04340,100.000,0.9784,True,13.410,0.05250,100.000,0.9582
1,\ADAUSDT\plots\2022-03-25T183319\backtest_resu...,5.5,ADAUSDT,True,-66.1700,-2.54000,6.000,0.0000,True,-71.440,-2.54000,5.962,0.9276
2,\ATOMUSDT\plots\2022-03-25T142413\backtest_res...,5.5,ATOMUSDT,True,22.9600,0.08870,100.000,0.9664,True,20.820,0.08110,100.000,0.9520
3,\ATOMUSDT\plots\2022-03-25T190936\backtest_res...,5.5,ATOMUSDT,True,-73.7200,-9.01000,5.970,0.0000,True,-74.540,-9.01000,5.994,0.9105
4,\AXSUSDT\plots\2022-03-25T094038\backtest_resu...,5.5,AXSUSDT,True,7.6130,0.03240,100.000,0.9368,True,7.707,0.03340,100.000,0.8438
5,\AXSUSDT\plots\2022-03-25T190311\backtest_resu...,5.5,AXSUSDT,True,-77.5700,-8.31000,5.986,0.0000,True,-76.160,-8.31000,5.993,0.8499
6,\BCHUSDT\plots\2022-03-19T132719\backtest_resu...,5.5,BCHUSDT,True,-2870.0000,1.49000,100.000,0.9597,True,-4800.000,1.59000,100.000,0.9580
7,\BCHUSDT\plots\2022-03-23T185443\backtest_resu...,5.5,BCHUSDT,True,0.3397,0.00180,100.000,0.9417,True,12.660,0.04340,100.000,0.9654
8,\BCHUSDT\plots\2022-03-25T182846\backtest_resu...,5.5,BCHUSDT,True,-63.7600,-1.95000,5.930,0.0000,True,-64.470,-1.95000,6.000,0.9332
9,\BNBUSDT\plots\2022-03-25T091844\backtest_resu...,5.5,BNBUSDT,True,3.8440,0.01730,100.000,0.9219,True,5.845,0.02190,100.000,0.9538


Now filter the dataframe

In [11]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]


allowed_symbols = ["EOS",
                       "XRP",
                       "BCH",
                       "LINK",
                       "ADA",
                       "DOT",
                       "DOGE",
                       "MATIC",
                       "BNB",
                       "THETA",
                       "AXS",
                       "LUNA",
                       "SAND",
                       "ATOM",
                       "RUNE",
                       "WAVES",]
allowed_symbols = [s + "USDT" for s in allowed_symbols]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['adgl_%'] > 0.04)
              #& (big_data['short'] == True)
              #& (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              #& (big_data['bnkrs_%'] > 95.0)
              #& (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
0,\ADAUSDT\plots\2022-03-24T141758\backtest_resu...,5.5,ADAUSDT,True,10.65,0.0434,100.0,0.9784,True,13.410,0.05250,100.000,0.9582,0.09590
2,\ATOMUSDT\plots\2022-03-25T142413\backtest_res...,5.5,ATOMUSDT,True,22.96,0.0887,100.0,0.9664,True,20.820,0.08110,100.000,0.9520,0.16980
6,\BCHUSDT\plots\2022-03-19T132719\backtest_resu...,5.5,BCHUSDT,True,-2870.00,1.4900,100.0,0.9597,True,-4800.000,1.59000,100.000,0.9580,3.08000
23,\LUNAUSDT\plots\2022-03-25T103120\backtest_res...,5.5,LUNAUSDT,True,25.07,0.0923,100.0,0.9449,True,6.458,-0.00595,100.000,0.9014,0.08635
25,\MATICUSDT\plots\2022-03-24T152614\backtest_re...,5.5,MATICUSDT,True,13.84,0.0560,100.0,0.9644,True,11.370,0.04640,100.000,0.9302,0.10240
29,\SANDUSDT\plots\2022-03-25T115948\backtest_res...,5.5,SANDUSDT,True,10.49,0.0431,100.0,0.9328,True,-94.060,-1.95000,5.965,0.0714,-1.90690
33,\WAVESUSDT\plots\2022-03-25T182205\backtest_re...,5.5,WAVESUSDT,True,15.01,0.0595,100.0,0.9467,True,12.580,0.00931,100.000,0.8947,0.06881
35,\XRPUSDT\plots\2022-03-19T120649\backtest_resu...,5.5,XRPUSDT,True,-649.10,0.7240,100.0,0.9594,True,-354.300,0.46200,100.000,0.8957,1.18600
36,\XRPUSDT\plots\2022-03-19T124422\backtest_resu...,5.5,XRPUSDT,True,-773.50,0.7670,100.0,0.9611,True,-422.200,0.43300,100.000,0.9439,1.20000
37,\XRPUSDT\plots\2022-03-19T142454\backtest_resu...,5.5,XRPUSDT,True,-476.40,0.5920,100.0,0.9605,True,-279.100,0.38900,100.000,0.9501,0.98100


In [10]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_706988/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
2,\ATOMUSDT\plots\2022-03-25T142413\backtest_res...,ATOMUSDT,0.16980,0.0887,0.08110,100.0,100.0,0.9664,0.9520
6,\BCHUSDT\plots\2022-03-19T132719\backtest_resu...,BCHUSDT,3.08000,1.4900,1.59000,100.0,100.0,0.9597,0.9580
23,\LUNAUSDT\plots\2022-03-25T103120\backtest_res...,LUNAUSDT,0.08635,0.0923,-0.00595,100.0,100.0,0.9449,0.9014
25,\MATICUSDT\plots\2022-03-24T152614\backtest_re...,MATICUSDT,0.10240,0.0560,0.04640,100.0,100.0,0.9644,0.9302
33,\WAVESUSDT\plots\2022-03-25T182205\backtest_re...,WAVESUSDT,0.06881,0.0595,0.00931,100.0,100.0,0.9467,0.8947
35,\XRPUSDT\plots\2022-03-19T120649\backtest_resu...,XRPUSDT,1.18600,0.7240,0.46200,100.0,100.0,0.9594,0.8957
36,\XRPUSDT\plots\2022-03-19T124422\backtest_resu...,XRPUSDT,1.20000,0.7670,0.43300,100.0,100.0,0.9611,0.9439
37,\XRPUSDT\plots\2022-03-19T142454\backtest_resu...,XRPUSDT,0.98100,0.5920,0.38900,100.0,100.0,0.9605,0.9501
